<h1><center>CMPE 462 - Project 2<br>Implementing an SVM Classifier<br>Due: May 18, 2020, 23:59</center></h1>

* **Student ID1:2016400198**
* **Student ID2:2016400126**
* **Student ID3:2018400279**

## Overview

In this project, you are going to implement SVM. For this purpose, a data set (data.mat) is given to you. You can load the mat dataset into Python using the function `loadmat` in `Scipy.io`. When you load the data, you will obtain a dictionary object, where `X` stores the data matrix and `Y` stores the labels. You can use the first 150 samples for training and the rest for testing. In this project, you will use the software package [`LIBSVM`](http://www.csie.ntu.edu.tw/~cjlin/libsvm/) to implement SVM. Note that `LIBSVM` has a [`Python interface`](https://github.com/cjlin1/libsvm/tree/master/python), so you can call the SVM functions in Python. 

## Task 1 - 30 pts

Train a hard margin linear SVM and report both train and test classification accuracy.

In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

from libsvm import svmutil as svm

In [ ]:
mat = scipy.io.loadmat('data.mat')

y_train = mat['Y'][:150,:].squeeze()
X_train = mat['X'][:150,:]

y_test = mat['Y'][150:,:].squeeze()
X_test = mat['X'][150:,:]
options = '-c 1e10'  # Very high C value imitates hard margin SVM
# Training Model
model = svm.svm_train(y_train, X_train, options)


In [ ]:
svm.svm_predict(y_test, X_test, model)
svm.svm_predict(y_train, X_train, model)

## Task 2 - 40 pts

Train soft margin SVM for different values of the parameter $C$, and with different kernel functions. Systematically report your results. For instance, report the performances of different kernels for a fixed $C$, then report the performance for different $C$ values for a fixed kernel, and so on.

In [ ]:
c_values = ['0.1','1','2','5','10']
t_values = {0:"Linear", 1:"Polynomial", 2:"RBS", 3:"Sigmoid"}

for c in c_values:
    for t in t_values.keys():
        options = "-t {} -c {}".format(t, c)
        model = svm.svm_train(y_train, X_train, options)
        print("Kernel:",t_values[t], '-- C =', c)
        pred_values, (acc,mse,scc), pred_values = svm.svm_predict(y_test, X_test, model)
        print('\n')

## Task 3 - 15 pts

Please report how the number of support vectors changes as the value of $C$ increases (while all other parameters remain the same). Discuss whether your observations match the theory.

In [ ]:
n_of_s_vectors = []
c_values = ['0.1','1','3','6','13','29','51','83','1e+2','231','1e+3',]
for c in c_values:
    options = "-t 3 -c {}".format(c)
    model = svm.svm_train(y_train, X_train, options)
    n_of_s_vectors.append(len(model.get_SV()))

c_values = list(map(float,c_values))
plt.plot(c_values,n_of_s_vectors)
plt.xlabel("penalty constant")
plt.ylabel("number of support vectors")
plt.show()

## Task 4 - 15 pts

Please investigate the changes in the hyperplane when you remove one of the support vectors, vs., one data point that is not a support vector.

In [ ]:
# main training model
options = "-t 0 -c 0.1"
main_model = svm.svm_train(y_train, X_train, options)

In [ ]:
index = main_model.get_sv_indices()[0]-1

sv_removed_x_train = np.concatenate([X_train[:index],X_train[index+1:]])
sv_removed_y_train = np.concatenate([y_train[:index],y_train[index+1:]])

model_sv_removed = svm.svm_train(sv_removed_y_train, sv_removed_x_train, options)

In [ ]:
# one non support vector removed model

index = -1
for i in range(len(X_train)):
    data = X_train[i,:]
    for sv in main_model.get_SV():
        if (list(data) != list(sv.values())[:-1]):
            index = i
            break
    if index != -1:
        break

non_sv_removed_x_train = np.concatenate([X_train[:index],X_train[index+1:]])
non_sv_removed_y_train = np.concatenate([y_train[:index],y_train[index+1:]])

model_non_sv_removed = svm.svm_train(non_sv_removed_y_train, non_sv_removed_x_train, options)

In [ ]:
svm.svm_predict(y_test, X_test, main_model)
svm.svm_predict(y_test, X_test, model_sv_removed)
svm.svm_predict(y_test, X_test, model_non_sv_removed)

In [ ]:
print("Main Model:", len(main_model.get_SV()))
print("SV removed Model:", len(model_sv_removed.get_SV()))
print("Non-SV removed Model:", len(model_non_sv_removed.get_SV()))

In [ ]:
# All support vectors of the main model and the model trained with one non-sv data point missing data are exactly the same.
main_model.get_SV() == model_non_sv_removed.get_SV()

In [ ]:
main_model_sv_arr = X_train[np.array(main_model.get_sv_indices())-1,:]
model_sv_removed_sv_arr = sv_removed_x_train[np.array(model_sv_removed.get_sv_indices())-1,:]
model_non_sv_removed_sv_arr = non_sv_removed_x_train[np.array(model_non_sv_removed.get_sv_indices())-1,:]

In [ ]:
main_model_weights = np.matmul(np.array(main_model.get_sv_coef()).T, main_model_sv_arr)
model_sv_removed_weights = np.matmul(np.array(model_sv_removed.get_sv_coef()).T, model_sv_removed_sv_arr)
model_non_sv_removed_weights = np.matmul(np.array(model_non_sv_removed.get_sv_coef()).T, model_non_sv_removed_sv_arr)

In [ ]:
print(main_model_weights)

In [ ]:
print(model_sv_removed_weights)

In [ ]:
print(model_non_sv_removed_weights)

As we can see, when we removed a non-sv data point, hyperplane did not change. On the contrary, removing sv data point changed the hyperplane, as expected.

### Bonus Task - 10 pts

Use Python and [CVXOPT](http://cvxopt.org) QP solver to implement the hard margin SVM. 